In [1]:
import pandas as pd
from glob import glob
def get_data_from_day(day):
    dataframes = []
    for f in glob(day+"/*.json"):
        dataframes.append(pd.read_json(f,lines=True))
    dataDay = pd.concat(dataframes)
    return(dataDay)
def extract_user_data(data):
    captured_keys = ['friends_count','favourites_count', 'description','screen_name', 'geo_enabled', 'lang', 'verified','time_zone','statuses_count', 'followers_count', 'following', 'name', 'location','created_at']
    users = set()
    userData = {}
    for key in captured_keys:
        userData[key] = []
    for ind,val in data.iterrows():
        if val['user']['name'] not in users:
            users.add(val['user']['name'])
            for key in captured_keys:
                userData[key].append(val['user'][key])
    userFrame = pd.DataFrame()
    for key in userData.keys():
        print(key,len(userData[key]))
        userFrame[key] = userData[key]
    return userFrame

def get_retweet_data(data_1):
    user_dict = {}
    follower_dict = {}
    account_creation_dict = {}
    location_dict = {}
    hashtag_dict = {}
    #if statement, to count of often a user is retweeted in the user_dict
    for retweeted in data_1['retweeted_status']:
        if retweeted['user']['name'] in user_dict.keys():
            user_dict[retweeted['user']['name']] += 1
        else:
            user_dict[retweeted['user']['name']] = 1 #also get the user's follower_count, created date, location, and used hasgtags
            follower_dict[retweeted['user']['name']] = retweeted['user']['followers_count']
            account_creation_dict[retweeted['user']['name']] = retweeted['user']['created_at']
            location_dict[retweeted['user']['name']] = retweeted['user']['location']
            hashtag_dict[retweeted['user']['name']] = retweeted['entities']['hashtags']


    retweet_data = pd.DataFrame()
    retweet_data['user'] = list(user_dict.keys())
    retweet_data['retweeted'] = list(user_dict.values())
    retweet_data['followers_count'] = list(follower_dict.values())
    retweet_data['created_at'] = list(account_creation_dict.values())
    retweet_data['location'] = list(location_dict.values())
    retweet_data['hastags'] = list(hashtag_dict.values())
    return retweet_data

def add_retweet_data(user_data,data_retweet_f):
    retweet_data_for_each_user = []
    for user in user_data['name']:
        if len(data_retweet_f[data_retweet_f['user'] == user]) !=0:
            retweet_data_for_each_user.append(data_retweet_f[data_retweet_f['user'] == user]['retweeted'].values[0])
        else:
            retweet_data_for_each_user.append(0)
    return retweet_data_for_each_user

def get_and_save_all_data(day):
    data = get_data_from_day(day)
    print("Finished getting Data")
    data_user = extract_user_data(data)
    print("Finished extracting Users")
    data_retweeted = get_retweet_data(data[pd.notnull(data['retweeted_status'])])
    print("Finished extracting retweet data")
    data_user['retweeted_count'] = add_retweet_data(data_user,data_retweeted)
    print("Finished adding retweet data to table")
    data_user.to_csv(day+"_userdata.csv")
    print("Saved File")

In [2]:
get_and_save_all_data("2016-06-21")

friends_count 206060
favourites_count 206060
description 206060
screen_name 206060
geo_enabled 206060
lang 206060
verified 206060
time_zone 206060
statuses_count 206060
followers_count 206060
following 206060
name 206060
location 206060
created_at 206060
